In [1]:
import pandas as pd

#Prepare data for models
from surprise import Dataset, Reader

#Seperate data into train and test set
from surprise.model_selection.split import train_test_split

#Model Selection
from surprise.prediction_algorithms.baseline_only import BaselineOnly as Baseline
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import SVD, SVDpp, NMF
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering

#Model selection Metric (RMSE)
from surprise.accuracy import rmse

In [2]:
df = pd.read_csv("/Users/grace/Documents/Recommender System/Data/DataSet2.csv", index_col=0)

/opt/miniconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df

,user_id,book_id,rating
0,2886,1,5
1,6158,1,5
2,3991,1,4
3,5281,1,5
4,5721,1,5
...,...,...,...
5916161,35336,10000,4
5916162,17999,10000,3
5916163,49007,10000,4
5916164,43319,10000,5


In [4]:
#Prepare data for models
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df, reader)

In [5]:
#create training set and test set
train_set, test_set = train_test_split(data, test_size=0.2, random_state=0)

In [6]:
#USED AS BASELINE AGAINST ALL OTHER MODELS
algo1 = Baseline()
algo1.fit(train_set)
prediction1 = algo1.test(test_set)
baseline_rmse = rmse(prediction1) #0.8541
print("Done!")

Estimating biases using als...
RMSE: 0.8541
Done!


In [7]:
#FOR KNN inspired algorithms
sim_options = {
    "name": "msd",
    "user_based": False,  # Compute  similarities between items (books as features), in an attempt to solve dimensionality due to there being fewer books than users
}

In [8]:
algo2 = KNNBasic(sim_options=sim_options)
algo2.fit(train_set)
prediction2 = algo2.test(test_set)
KNNBasic_rmse  = rmse(prediction2) #0.8631



Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8631


In [9]:
algo3 = KNNWithMeans(sim_options=sim_options)
algo3.fit(train_set)
predictions3 = algo3.test(test_set)
KNNWithMeans_rmse =rmse(predictions3)#0.8438

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8438


In [10]:
algo4 = KNNBaseline(sim_options=sim_options)
algo4.fit(train_set)
predicitions4 = algo4.test(test_set)
KNNBaseline_rmse = rmse(predicitions4)#0.8401

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8401


In [11]:
algo5 = KNNWithZScore(sim_options=sim_options)
algo5.fit(train_set)
predictions5 = algo5.test(test_set)
KNNWithZScore_rmse = rmse(predictions5) #0.8457

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8457


Matrix Factorization was considered to solve the issues I've been having with dimensionality

In [12]:
algo6 =SVD() 
algo6.fit(train_set)
predictions6 = algo6.test(test_set)
SVD_rmse = rmse(predictions6) #0.8292

RMSE: 0.8292


In [14]:
algo8 = NMF()
algo8.fit(train_set)
predictions8 = algo8.test(test_set)
NMF_rmse = rmse(predictions8) #0.8622

RMSE: 0.8622


In [22]:
algorithms = ["Baseline", "KNNBasic","KNNWithMean", "KNNBaseline", "KNNWithZScore", "SVD", "NMF"]
rmse = [baseline_rmse, KNNBasic_rmse, KNNWithMeans_rmse, KNNBaseline_rmse, KNNWithZScore_rmse, SVD_rmse, NMF_rmse]

data_rmse = dict(zip(algorithms,rmse))
pd.DataFrame.from_dict(data_rmse, orient='index',columns=["RMSE"]).sort_values(by='RMSE')

,RMSE
SVD,0.829188
KNNBaseline,0.840139
KNNWithMean,0.843838
KNNWithZScore,0.845705
Baseline,0.854075
NMF,0.862186
KNNBasic,0.863087
